# Import python libraries

In [1]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import confusion_matrix, plot_confusion_matrix

# Import CountVectorizer and TFIDFVectorizer from feature_extraction.text.
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

### Import Train and Test Data

In [2]:
X_train= pd.read_csv("./datasets/X_train.csv" , squeeze = True)
y_train= pd.read_csv("./datasets/y_train.csv", squeeze = True)

X_test= pd.read_csv("./datasets/X_test.csv" , squeeze = True)
y_test= pd.read_csv("./datasets/y_test.csv", squeeze = True)

### Model 7: Log Regression : Pipeline - GridSearch

In [3]:
# Let's set a pipeline up with two stages:
# 1. CountVectorizer (transformer)
# 2. Logistic Regression (estimator)
pipe = Pipeline([
    ('cvec', CountVectorizer()),
    ('lr', LogisticRegression())
])

In [4]:
# Search over the following values of hyperparameters:
# Maximum number of features fit: 2000, 3000, 4000, 5000
# Minimum number of documents needed to include token: 2, 3
# Maximum number of documents needed to include token: 90%, 95%
# Check (individual tokens) and also check (individual tokens and 2-grams).
pipe_params = {
    'cvec__max_features' : [200, 300, 400, 500],
    'cvec__stop_words' : [None,['Sherlock', 'Poirot'], 'english'],
    'cvec__ngram_range' : [(1,1), (1,2)],
    'lr__penalty':['l1', 'l2'],
    'lr__C':[1,0.1,0.05]
}

In [5]:
# Instantiate GridSearchCV.
gs_lr = GridSearchCV(pipe,
                      pipe_params,
                      cv = 5,
                      verbose=1,
                      n_jobs=8)

In [6]:
# Fit GridSearch to training data.
gs_lr.fit(X_train, y_train)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    1.4s
[Parallel(n_jobs=8)]: Done 560 tasks      | elapsed:    7.6s
[Parallel(n_jobs=8)]: Done 720 out of 720 | elapsed:    9.8s finished


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('cvec', CountVectorizer()),
                                       ('lr', LogisticRegression())]),
             n_jobs=8,
             param_grid={'cvec__max_features': [200, 300, 400, 500],
                         'cvec__ngram_range': [(1, 1), (1, 2)],
                         'cvec__stop_words': [None, ['Sherlock', 'Poirot'],
                                              'english'],
                         'lr__C': [1, 0.1, 0.05], 'lr__penalty': ['l1', 'l2']},
             verbose=1)

In [7]:
gs_lr_df = pd.DataFrame(gs_lr.cv_results_)

In [8]:
gs_lr.best_estimator_

Pipeline(steps=[('cvec',
                 CountVectorizer(max_features=500, stop_words='english')),
                ('lr', LogisticRegression(C=1))])

In [9]:
gs_lr.best_params_

{'cvec__max_features': 500,
 'cvec__ngram_range': (1, 1),
 'cvec__stop_words': 'english',
 'lr__C': 1,
 'lr__penalty': 'l2'}

### Run the model on Test data

In [10]:
# Evaluate the best fit model on the test data.
best_lr = gs_lr.best_estimator_
print(f"Training Score from best Logistic Regression Classifier: {gs_lr.score(X_train , y_train)}")
print(f"Test Score from best Logistic Regression Classifier: {gs_lr.score(X_test, y_test)}") 

Training Score from best Logistic Regression Classifier: 0.9923780487804879
Test Score from best Logistic Regression Classifier: 0.9236641221374046
